In [ ]:
import os
import sys
import datetime
import numpy as np
import matplotlib.pyplot as plt
import random
import uuid

from tqdm import tqdm

In [ ]:
from IPython.display import clear_output

In [ ]:
data_root = "../dataset"
images_root = os.path.join(data_root, "images_all")
masks_root = os.path.join(data_root, "masks_all")

In [ ]:
test_data_dist = os.path.join(data_root, "test_data.csv")

In [ ]:
scripts_path = "../scripts"

In [ ]:
sys.path.append(scripts_path)

In [ ]:
import torch
import constants as const

from train_utils import *
from metrics import get_iou_metric

In [ ]:
test_dataset = get_test_dataset_segmentation(test_data_dist, images_root, masks_root)

In [ ]:
model = torch.load("../models/segmentation_model_xception_backbone.pth")
model.eval()

In [ ]:
test_loader = get_data_loader(test_dataset, batch_size = const.batch_size_val, shuffle=False, num_workers = 0)

In [ ]:
test_loader_iterator = test_loader._get_iterator()

In [ ]:
def iou_numpy(outputs: np.array, labels: np.array):
    intersection = np.logical_and(labels, outputs)
    union = np.logical_or(labels, outputs)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [ ]:
preds = []
gts = []
ious = []
idx = 0
results = {}

try:
    with torch.no_grad(): 
        current_sample = test_loader_iterator.next()
            
        while current_sample is not None:
            if idx <= 1250:
                clear_output(wait=True)
                idx += 1
                image, gt = current_sample
                image, gt = image.cuda(), gt.cuda()

                # Perform prediction            
                pred = model(image)

                # Transform outputs
                pred_cpu = pred.cpu().numpy()
                gt_cpu = gt.cpu().numpy()

                # Post-process the results
                pred_cpu[pred_cpu >= 0.5] = 1
                pred_cpu[pred_cpu < 0.5] = 0

                pred_cpu = pred_cpu.astype(int)
                gt_cpu = gt_cpu.astype(int)

                iou = iou_numpy(pred_cpu, gt_cpu)
                print("Processing image {} / {}. IoU: {}".format(idx, 1300, iou))
                ious.append(iou)

                # Save results
                gts.append(gt_cpu)
                preds.append(pred_cpu)
                
                results[idx] = (image.cpu().numpy(), pred_cpu, gt_cpu, iou)

                current_sample = test_loader_iterator.next()
            else:
                break
except StopIteration as ex:
    pass

In [ ]:
ious_arr = np.array(ious)
ious_arr = ious_arr[ious_arr > 0]
print("Average IoU (no outliers): {}".format(np.mean(ious_arr)))

In [ ]:
def show_pred_gt(image, pred, gt, iou, save_name = None):

    f, axarr = plt.subplots(1,3, figsize = (16, 8))
    axarr[0].imshow(image[0, :, :, :].transpose(2, 1, 0))
    axarr[1].imshow(pred[0, :, :, :].transpose(2, 1, 0), cmap = 'gray')
    axarr[2].imshow(gt[0, :, :, :].transpose(2, 1, 0), cmap = 'gray')
    
    axarr[0].title.set_text("Input")
    axarr[1].title.set_text("Prediction. IOU: {:.2f}".format(iou))
    axarr[2].title.set_text("Ground Truth")
    
    if save_name is not None:
        plt.savefig(os.path.join("../outputs", save_name), bbox_inches = "tight")

In [ ]:
index = random.randint(0, len(results))
print(index)
image, pred, gt, iou = results[index]

show_pred_gt(image, pred, gt, iou, save_name = "{}_{}".format(index, str(uuid.uuid4())))